In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.metrics import mse
import matplotlib.pyplot as plt
from keras.backend import set_session
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.models import Model
import numpy as np
from keras.layers import Conv2D, BatchNormalization, ZeroPadding2D, UpSampling2D, Input, Lambda, multiply, Input, concatenate
from keras.layers.advanced_activations import LeakyReLU
from PIL import Image
import tensorflow as tf


In [ ]:
def ConvertImage(image, dim=32):
    x_cut = image.size[0] - image.size[0] % dim
    y_cut = image.size[1] - image.size[1] % dim

    cropBox = []
    cropBox.append(int((image.size[0] - x_cut) / 2))
    cropBox.append(int((image.size[1] - y_cut) / 2))
    cropBox.append(int((image.size[0] + x_cut) / 2))
    cropBox.append(int((image.size[1] + y_cut) / 2))

    img_cropped = image.crop(cropBox)
    img_cropped = img_to_array(img_cropped)
    img_cropped = (np.expand_dims(img_cropped, axis=0).astype('float32'))/255
    return img_cropped


def add_noise(image, sigma):
    noise = np.clip(image + np.random.normal(scale=sigma, size=image.shape), 0, 1).astype(np.float32)
    return noise

In [ ]:
class Hourglass:
    def __init__(self, in_size, in_channel, out_channel, nu, nd, ns, ku,kd, ks,upsample='nearest'):
        self.in_size = in_size
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.nu = nu
        self.nd = nd
        self.ns = ns
        self.ku = ku
        self.kd = kd
        self.ks = ks
        self.upsample = upsample
        self.use_bias = True

    def up_sample(self, x, size=2):
        index = x.shape
        new_w = int(round(index[1] * size))
        new_h = int(round(index[2] * size))
        return tf.image.resize(x, [new_w, new_h], method=tf.image.ResizeMethod.BILINEAR);

    def down(self, x, n, k, use_bias=True):
        pad_size = int((k - 1) / 2)
        x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
        x = Conv2D(n, k, strides=2, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

        x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
        x = Conv2D(n, k, strides=1, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)
        return x

    def skip(self, x, n, k, use_bias=True):
        pad_size = int((k - 1) / 2)
        x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
        x = Conv2D(n, k, strides=1, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)
        return x

    def up(self, x, n, k, upsample_mode, use_bias=True):
        x = BatchNormalization()(x)
        pad_size = int((k - 1) / 2)
        x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
        x = Conv2D(n, k, strides=1, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

        x = Conv2D(n, 1, strides=1, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

        x = self.up_sample(x, 2)
        return x

    def model_structure(self):
        width, height = self.in_size
        input = Input((width, height, self.in_channel))
        x = input
        n = 5
        skips = []
        for i in range(n):
            x = self.down(x, self.nd[i], self.kd[i], self.use_bias)
            if self.ns[i] != 0:
                skip = self.skip(x, self.ns[i], self.ks[i], self.use_bias)
                skips.append(skip)

        x = self.up(x, self.nu[n - 1], self.ku[n - 1], self.upsample, self.use_bias)
        skips = skips[::-1]
        for i in range(n - 1):
            j = (n - 1)- i 
            if i != 0:
                x = concatenate([x, skips[i+1]], axis=-1)
            x = self.up(x, self.nu[j-1], self.ku[j-1],self.upsample, self.use_bias)

        output = Conv2D(self.out_channel, 1, activation='sigmoid')(x)

        model = Model(inputs=input, outputs=output)
        return model



In [ ]:

## environment options
image_path = 'F16_GT.png'

## experiment options
sigma = 1. / 30
method = 'random'
input_channel = 3
lr = 0.01
num_iter = 5000

### load data
img = load_img(image_path)
x = ConvertImage(img)
x = add_noise(x, 25 / 255.)
array_to_img(x[0]).save('original.png')
    

### build code z
b, w, h, c = x.shape

### build model
layers = Hourglass((w, h), input_channel, c,
                      nu=[128, 128, 128, 128, 128],
                      nd=[128, 128, 128, 128, 128],
                      ns=[4, 4, 4, 4, 128],
                      ku=[3, 3, 3, 3, 3],
                      kd=[3, 3, 3, 3, 3],
                      ks=[1, 1, 1, 1, 1],
                      upsample='bilinear'
                      )

model = layers.model_structure()
model.compile(optimizer=Adam(lr=lr), loss=mse)
model.summary()


In [ ]:
#z = make_noise(method, input_channel, (w, h))
z = np.random.uniform(0, 0.1, size=(1,w,h,input_channel))
losses = []
for i in range(num_iter+1):
    temp = z + np.random.normal(0, sigma, size = z.shape)
    loss = model.train_on_batch(temp, x)
    if i % 100 == 0:
        print('iteration no.: %d ,loss : %f' % (i, loss))
        y = model.predict_on_batch(z)
        array_to_img(y[0]).save('%d.png' % i)

